In [1]:
import sys, os, random
sys.path.insert(0,os.path.expanduser('~/git/pyzx')) # git version
sys.path.insert(0,'/workspaces/pyzx')
import pyzx as zx
from pyzx.pauliweb import PauliWeb, compute_pauli_webs
zx.settings.colors = zx.original_colors

This notebook demonstrates using PyZX's methods for automatically computing and drawing bounded Pauli webs. A Pauli web (a.k.a. _correlation surface_) is a labelling of the edges of ZX-diagram from the set $\{I, X, Y, Z\}$ with certain properties. To explain these properties, it is useful to make some definitions.

To account for Hadmard edges, we label "half-edges" of a diagram. For a pair of connected vertices `v`, `w`, we represent the half-edge closest to `v` as `(v,w)` and the half-edge closest to `w` as `(w,v)`.

We say a spider `v` is _stabilised_ by a Pauli web if `v` (considered as a state) is a +1 eigenstate of the Pauli string labelling its adjacent half-edges and we say `v` is _anti-stabilised_ by a Pauli web if it is a -1 eigenstate of its adjacent half-edges.

A Pauli web is said to be _closed_ if:
1. every spider `v` in the ZX-diagram is stabilised or anti-stabilised
2. for every half-edge `(v,w)` labelled by a Pauli $P$, the other half-edge `(w,v)` is labelled by $P$ for a simple edge and $HPH$ for a Hadamard edge

Otherwise, we say a Pauli web is _open_. The spiders and edges violating conditions 1 and 2 above are called the _boundary_ of a Pauli web.

The function `compute_pauli_webs` automitically associates each non-input vertex `v` in the diagram to an integer `order[v]` giving a time-ordering, and one or two bounded Pauli webs, with the following properties:
1. the boundary of the web consists of only `v` itself, earlier non-Clifford spiders `w` (`order[v] < order[w]>`), and inputs
2. Z spiders have a web `zweb[v]` with a Z-colored edge incident to `v`
3. X spiders have a web `xweb[v]` with a X-colored edge incident to `v`
4. output vertices have two webs `zweb[v]` and `xweb[v]` corresponding to both colors at `v`

Under the hood, this is using PyZX's gflow-finding algorithm to compute a focussed Pauli flow and translate this data into Pauli webs.

# CNOT Examples

In [2]:
# Here's a simple example of a single CNOT gate

c = zx.qasm("""
qreg q[2];
cx q[0], q[1];
""")
g = c.to_graph()
order, zwebs, xwebs = compute_pauli_webs(g)
zx.draw(g, labels=True)


In [3]:
# There are 4 Pauli webs corresponding to the two outputs
zx.draw(g, labels=True, pauli_web=xwebs[4])
zx.draw(g, labels=True, pauli_web=xwebs[5])
zx.draw(g, labels=True, pauli_web=zwebs[4])
zx.draw(g, labels=True, pauli_web=zwebs[5])

In [4]:
# Pauli webs are computed in the backwards (i.e. Heisenberg-style) by default,
# propegating Paulis from outputs to inputs. Paulis can be propegated forward
# instead by passing backwards=False.

order, zwebsf, xwebsf = compute_pauli_webs(g, backwards=False)
zx.draw(g, labels=True, pauli_web=xwebsf[0])
zx.draw(g, labels=True, pauli_web=xwebsf[1])
zx.draw(g, labels=True, pauli_web=zwebsf[0])
zx.draw(g, labels=True, pauli_web=zwebsf[1])

In [5]:
# Here's an example with more CNOT gates

c = zx.qasm("""
qreg q[3];
cx q[0], q[1];
cx q[1], q[2];
cx q[0], q[2];
cx q[2], q[1];
cx q[0], q[1];
cx q[2], q[0];
""")
g = c.to_graph()

zx.draw(g, labels=True)

In [6]:
order, zwebs, xwebs = compute_pauli_webs(g)
zx.draw(g, labels=True, pauli_web=xwebs[15])
zx.draw(g, labels=True, pauli_web=zwebs[15])
zx.draw(g, labels=True, pauli_web=xwebs[16])
zx.draw(g, labels=True, pauli_web=zwebs[16])
zx.draw(g, labels=True, pauli_web=xwebs[17])
zx.draw(g, labels=True, pauli_web=zwebs[17])

In [7]:
# Alternatively, we can fuse all the spiders of the same color together to get a
# more interesting diagram.
zx.spider_simp(g)
zx.draw(g, labels=True)

In [8]:
# Since this describes the same unitary, the 6 output webs should all have the same support on
# the inputs as before.
order, zwebs, xwebs = compute_pauli_webs(g)
zx.draw(g, labels=True, pauli_web=xwebs[15])
zx.draw(g, labels=True, pauli_web=zwebs[15])
zx.draw(g, labels=True, pauli_web=xwebs[16])
zx.draw(g, labels=True, pauli_web=zwebs[16])
zx.draw(g, labels=True, pauli_web=xwebs[17])
zx.draw(g, labels=True, pauli_web=zwebs[17])

# Clifford examples

In [9]:
# Next, we'll look at a single H gate
c = zx.qasm("""
qreg q[1];
h q[0];
""")
g = c.to_graph()

# PyZX renders this as a single H-edge connected to an identity spider
zx.draw(g)

# we don't really need this id-spider, so we can remove it with id_simp
zx.id_simp(g)
zx.draw(g, labels=True)

order, zwebs, xwebs = compute_pauli_webs(g)

In [10]:
# The single output has two Pauli webs
zx.draw(g, labels=True, pauli_web=zwebs[2])
zx.draw(g, labels=True, pauli_web=xwebs[2])

# Note that on a hadamard edge, the color changes in the middle of the edge. To handle this, Pauli webs actually label every
# "half-edge" with a Pauli. (v,w) means the half of the edge touching v, whereas (w,v) means the half of the edge touching w.

#  See for example:
print(zwebs[2].half_edges())

{(0, 2): 'X', (2, 0): 'Z'}


In [11]:
# Here's an example mixing CNOT and H gates

c = zx.qasm("""
qreg q[3];
cx q[0], q[1];
h q[1];
h q[2];
cx q[1], q[2];
cx q[0], q[2];
h q[0];
cx q[2], q[1];
h q[1];
h q[1];
cx q[0], q[1];
cx q[2], q[0];
""")
g = c.to_graph()

zx.draw(g, labels=True)

In [12]:
# Note the we get some Y-edges appear, which are shown in a third color
order, zwebs, xwebs = compute_pauli_webs(g)
zx.draw(g, labels=True, pauli_web=xwebs[20])
zx.draw(g, labels=True, pauli_web=zwebs[20])
zx.draw(g, labels=True, pauli_web=xwebs[21])
zx.draw(g, labels=True, pauli_web=zwebs[21])
zx.draw(g, labels=True, pauli_web=xwebs[22])
zx.draw(g, labels=True, pauli_web=zwebs[22])

In [13]:
# Next, we'll look at a single S gate
c = zx.qasm("""
qreg q[1];
s q[0];
""")
g = c.to_graph()
zx.draw(g, labels=True)

order, zwebs, xwebs = compute_pauli_webs(g)
zx.draw(g, labels=True, pauli_web=xwebs[2])

In [14]:
# Since we can handle CNOT, H, and S, we can now compute Pauli webs for arbitrary Clifford
# unitaries.

random.seed(1337)
c = zx.generate.CNOT_HAD_PHASE_circuit(qubits=4, depth=25, clifford=True)
g = c.to_graph(compress_rows=False)
zx.draw(g, labels=True)

order, zwebs, xwebs = compute_pauli_webs(g)
zx.draw(g, pauli_web=zwebs[41])
zx.draw(g, pauli_web=xwebs[41])

In [15]:
# As before, we can simplify the diagram without changing how Pauli webs behave on the boundaries
g1 = g.copy()

# Calling full_reduce will turn any Clifford circuit into GSLC form, consisting of an input row
# of Clifford Z-spiders connected to an output row of Clifford Z-spiders via Hadamard edges.
# Inputs and outputs can be Hadamard or normal edges.
zx.full_reduce(g1)
g1.normalize()

# nudge some nodes to left, for visibility
for v in [0,1,2,3,11]: g1.set_row(v, g1.row(v)-1)
zx.draw(g1, labels=True)

order, zwebs, xwebs = compute_pauli_webs(g1)

zx.draw(g1, pauli_web=zwebs[41])
zx.draw(g1, pauli_web=xwebs[41])

# Errors and corrections

Before moving on to Clifford+T examples, we'll show how to introduce errors into ZX-diagrams, correct them, and prove correctness.

We can use the `PauliWeb` class to represent arbitrary Pauli errors on half edges. Note this will _not_ usually be a valid, closed Pauli web, but we can still use the `PauliWeb` class to hold this data.

In [26]:
random.seed(1337)
g = zx.generate.CNOT_HAD_PHASE_circuit(qubits=4, depth=10, clifford=True).to_graph(compress_rows=False)
zx.draw(g, labels=True)

# introduce random Pauli errors on edges and draw them
errors = PauliWeb(g)
for e in g.edges():
    s,t = g.edge_st(e)
    if random.random() > 0.75:
        errors.add_half_edge((s,t), random.choice(['X','Y','Z']))
    if random.random() > 0.75:
        errors.add_half_edge((t,s), random.choice(['X','Y','Z']))
zx.draw(g, pauli_web=errors)

In [32]:
# To find the appropriate corrections, we should introduce Pauli X and Z gates at the outputs
# to make all 8 output webs commute with `errors`.

order, zwebs, xwebs = compute_pauli_webs(g)
output_es = [(o, next(iter(g.neighbors(o)))) for o in g.outputs()]


errors1 = errors.copy()

# for each of the outputs, if the errors anti-commute with the associated Z-web, introduce
# an X-error at that output to make it commute, and do similar for X-webs.
for o,n in output_es:
    print(f'Z-web {o}: ', end='')
    if zwebs[o].commutes_with(errors):
        print('commutes')
    else:
        print('anti-commutes', end='')
        errors1.add_half_edge((o,n), 'X')
        if zwebs[o].commutes_with(errors1):
            print(', corrected')
    print(f'X-web {o}: ', end='')
    if xwebs[o].commutes_with(errors):
        print('commutes')
    else:
        print('anti-commutes', end='')
        errors1.add_half_edge((o,n), 'Z')
        if xwebs[o].commutes_with(errors1):
            print(', corrected')

Z-web 20: commutes
X-web 20: anti-commutes, corrected
Z-web 21: commutes
X-web 21: commutes
Z-web 22: anti-commutes, corrected
X-web 22: commutes
Z-web 23: commutes
X-web 23: commutes


# Clifford+T Examples

In [18]:
# Generate a random CNOT, H, T circuit
random.seed(1330)
c = zx.generate.CNOT_HAD_PHASE_circuit(qubits=3, depth=40)
# for g in c.gates: print(g)
zx.draw(c)

In [19]:
# Convert to a ZX diagram and call the full_reduce procedure on it (PyZX's main ZX diagram optimisation pass)
g = c.to_graph()
zx.full_reduce(g)
zx.to_rg(g)

# Normalise compacts the circuit visually and ensures every input/output is connected to a Z spider
g.normalize()

# Compute the time-ordering on nodes (which is only important for the non-Clifford nodes) and compute the Pauli
# webs for every node.
order, zwebs, xwebs = compute_pauli_webs(g)

# Draw the simplified ZX diagram. Note blue edges correspond to edges with Hadamard gates
zx.draw(g, labels=True)

In [20]:
pw = zwebs[43]
pw.half_edges()

{(3, 1): 'Y',
 (1, 3): 'Y',
 (3, 43): 'Z',
 (43, 3): 'Z',
 (3, 5): 'Y',
 (5, 3): 'Y',
 (3, 58): 'Z',
 (58, 3): 'X',
 (3, 54): 'Z',
 (54, 3): 'Z',
 (3, 14): 'Y',
 (14, 3): 'Y',
 (5, 16): 'Y',
 (16, 5): 'Y',
 (5, 2): 'Y',
 (2, 5): 'Y',
 (5, 43): 'X',
 (43, 5): 'Z',
 (5, 58): 'X',
 (58, 5): 'X',
 (5, 54): 'X',
 (54, 5): 'Z',
 (14, 58): 'X',
 (58, 14): 'X',
 (14, 16): 'Y',
 (16, 14): 'Y',
 (16, 43): 'Z',
 (43, 16): 'Z',
 (16, 58): 'Z',
 (58, 16): 'X'}

In [21]:
# Once the Pauli webs have been computed, a specific web can be highlighted by `zx.draw` by passing it in as
# an optional argument. Note that webs change color when they cross Hadamard edges.
zx.draw(g, labels=True, pauli_web=zwebs[43])

We now show how this works in some simpler cases. The first is a single T gate.

The T gate becomes a single, 1-legged phase gadget, connected to the input. This can be implemented by Z-merging a T magic state, then doing either an X or a Y measurement, depending on the parity of the Pauli web.

In [22]:
c = zx.qasm("""
qreg q[1];
t q[0];
""")
zx.draw(c)

g = c.to_graph()
zx.full_reduce(g)
order, zwebs, xwebs = compute_pauli_webs(g)

# highlight the web associated to the T spider
zx.draw(g, labels=True, pauli_web=zwebs[1])

The next example is a circuit with some CNOT gates and a T gate, which can be simplified to a single phase gadget. I'm doing this manually here, since the automated simplifier comes up with a different answer (which is equivalent to this one, up to local Cliffords, but less clear what is going on).

In [23]:
c = zx.qasm("""
qreg q[3];
cx q[0], q[1];
cx q[1], q[2];
t q[2];
cx q[1], q[2];
cx q[0], q[1];
""")
zx.draw(c)

# manual ZX simplification to get a single phase gadget
g = c.to_graph()
zx.simplify.gadgetize(g, graphlike=False)
zx.basicrules.strong_comp(g, 5, 7)
zx.simplify.spider_simp(g, quiet=True)
zx.basicrules.strong_comp(g, 3, 6)
zx.simplify.spider_simp(g, quiet=True)
zx.simplify.id_simp(g, quiet=True)

zx.draw(g, labels=True)

d = dict()

# pauli web calculation
order, zwebs, xwebs = compute_pauli_webs(g, debug=d)

# highlight the web associated to the T spider
zx.draw(g, labels=True, pauli_web=zwebs[15])